In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
!pip install tifffile


     |████████████████████████████████| 133kB 5.0MB/s 


In [3]:
import numpy as np
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter
import random
import tifffile as tiff
import cv2
from keras.layers import Concatenate
import os

Using TensorFlow backend.


In [0]:
"""
:Date: 2019-08-05
:Version: 1
:Authors:
    - Varun Vallabhan

This script is needed to create augmented images from a given set of images for 
creating a training dataset for ADDN and UNET. This is the updated version which 
not only uses the tiff file but also, can use other file formats like .bmp,.png,.jpg.


"""






def load_images(folder):
    """
    :folder: string
    :rtype: List
    This is the load function for images which are not in .tif file format. The function 
    load the the images in a list and and returns the list.
    """
    images=[]
    for filename in sorted(os.listdir(folder)):
        
        img = cv2.imread(os.path.join(folder,filename),0)
        
        if img is not None:
            images.append(img)
    return images

  
def elastic_transform(image, alpha, sigma, alpha_affine, random_state=None):
    """
    :image: numpy array
    :alpha: numpy array
    :sigma: numpy array
    :alpha_affine: numpy array
    
    This function adds elastic transforms to the images. This uses affine transform to reorient the image to create the elastic effect.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    shape_size = shape[:2]

    # Random affine
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32(
        [center_square + square_size, [center_square[0] + square_size, center_square[1] - square_size],
            center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)
    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)

    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dz = np.zeros_like(dx)

    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
    indices = np.reshape(y + dy, (-1, 1)), np.reshape(x + dx, (-1, 1)), np.reshape(z, (-1, 1))

    return map_coordinates(image, indices, order=1, mode='reflect').reshape(shape)
  
  
def random_transforms(items, nb_min=0, nb_max=5, rng=np.random):
    """
    :items: numpy array
    :nb_min: int
    :nb_max: int
    :rng: int
    :rtype: numpy array
    These are a bunch of transforms like rotating flipping which are used in augment function.
    """
    all_transforms = [
        lambda x: x,
        lambda x: np.fliplr(x),
        lambda x: np.flipud(x),
        lambda x: np.rot90(x, 1),
        lambda x: np.rot90(x, 2),
        lambda x: np.rot90(x, 3),
        # lambda x: add_noise(x),
    ]
    n = rng.randint(nb_min, nb_max + 1)
    items_t = [item.copy() for item in items]
    for _ in range(n):
        idx = rng.randint(0, len(all_transforms))
        transform = all_transforms[idx]
        items_t = [transform(item) for item in items_t]
    return items_t
  
  
def add_noise(img):
    """
    :img: numpy array
    :rtype: numpy array
    Adds noise to images at random pixels.
    """
    for i in range(200):
        temp_x = np.random.randint(0,img.shape[0])
        temp_y = np.random.randint(0,img.shape[1])
        img[temp_x][temp_y] = 255
    return img

  
def normalization(imgs):
    """
    :imgs:numpy array
    :rtype: numpy array
    Normalization of the images
    """
    _mean, _std = np.mean(imgs), np.std(imgs)
    normalize = lambda x: (x - _mean) / (_std + 1e-10)
    return normalize(imgs)
  
  
def tiff_img_reader(imgs_path,msks_path = None):
    """
    :imgs_path: string
    :msks_path: None in case of test/ string in case of train
    :rtype: numpy array, numpy array (float32)
    
    tiff img reader segment to read tiff files. Requires tifffile library.
    """
    imgs = tiff.imread(imgs_path).astype('float32') /255
    if msks_path != None:
        msks =tiff.imread(msks_path) // 255
    else:
        msks = None
    return imgs,msks

  
def data_split(imgs_path,msks_path,tiff_file=True):
    """
    :imgs_path: string
    :msks_path: string
    :tiff_file: bool
    :rtype: numpy array,numpy array,numpy array,numpy array (float32)
    
    Splits data into train and validation before it is sent for augmentation.
    """
    if tiff_file==True:
        imgs, msks =  tiff_img_reader(imgs_path,msks_path)
    else:
        imgs=np.asarray(load_images(imgs_path))/255.
        msks=np.asarray(load_images(msks_path))/255.
    idxs = random.sample(range(imgs.shape[0]),imgs.shape[0] // 5)
    rest = [i for i in range(imgs.shape[0]) if i not in idxs]
    imgs_val = imgs[idxs]
    msks_val = msks[idxs]
    imgs_trn = imgs[rest]
    msks_trn = msks[rest]
    return imgs_trn,msks_trn,imgs_val,msks_val

  
def Augment(imgs, msks, output_shape, aug_ration=10, full_size=False,transform = False, rng=np.random):
    
    '''
    :imgs: float
    :msks: uint8
    :output_shape:(int,int)
    :aug_ration: int
    :full_size: bool
    :transform:bool
    :rng:int
    :rtype: numpy array,numpy array (float32)
    
    The images are first cropped to desired shape and size accoring to the output shape requested 
    and then they are subjected to elastic transformationa and then they are randomly flipped or 
    rotated using the random transform function. If the images are not to be cropped set full_size to False which is the default
    and if only elastic transformations are required set the transform to False which will prevent random transformation. aug_ration 
    decides the number of augmented images per actual image. 
    '''
    img_len, H, W = imgs.shape
    wdw_H,wdw_W = output_shape
    img_batch = np.zeros((img_len*aug_ration,) + output_shape, dtype=np.float32)
    msk_batch = np.zeros((img_len*aug_ration,) + output_shape, dtype=np.uint8)
    batch_idx = 0
    for img_idx in range(img_len):
        print('original image number:%d output shape:(%d,%d)'%(img_idx,wdw_H,wdw_W))
        for num in range(aug_ration):
            if full_size:
              im = imgs[img_idx]
              im_mask = msks[img_idx]
            else:
              
              y0, x0 = rng.randint(0, H - wdw_H), rng.randint(0, W - wdw_W)
              
              y1, x1 = y0 + wdw_H, x0 + wdw_W
              
              im = imgs[img_idx][y0:y1, x0:x1]
              im_mask = msks[img_idx][y0:y1, x0:x1]
            #### elastic  transform
            if np.random.randint(0, 10) > 7:
                im_merge = np.concatenate((im[..., None], im_mask[..., None]), axis=2)
                im_merge_t = elastic_transform(im_merge, im_merge.shape[1] * 2, im_merge.shape[1] * 0.08,im_merge.shape[1] * 0.08)
                im_t = im_merge_t[..., 0]
                im_mask_t = im_merge_t[..., 1]
                img_batch[batch_idx] = im_t
                msk_batch[batch_idx] = im_mask_t
            else:
                img_batch[batch_idx] = im
                msk_batch[batch_idx] = im_mask
            if transform:
                [img_batch[batch_idx], msk_batch[batch_idx]] = random_transforms(
                    [img_batch[batch_idx], msk_batch[batch_idx]])
            batch_idx += 1
    img_batch = img_batch[:, :, :, np.newaxis]
    msk_batch = msk_batch[:, :, :, np.newaxis]
    # tiff.imsave('test.tif',np.uint8(img_batch[0]*255))
    return img_batch,msk_batch
  
  
def save_npy(data, name, npy_path):
    """
    :data: numpy array
    :name: string
    :npy_path: string
    
    This saves the augmented images in a numpy file.
    """
    print ("saving npy data...")
    np.save(npy_path+'/'+name, data)
    print ("Data %s saved in root: %s." % (name, npy_path))
    
    
def generate_test(path,tiff_file=True,height=128,width=128):
  """
  :path: string
  :tiff_file: bool
  :height: int
  :width: int
  :rtype: numpy array
  
  This generates the test image of the requested size.
  """
  if tiff_file:
    imgs_tst,_ = tiff_img_reader(path)
  else:
    imgs_tst=np.asarray(load_images(path))/255. 
  imgs_tst = imgs_tst[:,:height,:width,np.newaxis]  
  return imgs_tst
  
  
def main(config):
  """
  :config: dict
  
  This funtions calls all the funtions in order to read, split, augment and save the data. 
  the parameters are passed in a config dictionary which includes the paths and modes that are needed for generating the images 
  """
  imgs_trn, msks_trn, imgs_val, msks_val = data_split(config['train_image_path'],config['train_mask_path'],tiff_file=config["tiff_file"])
  print('generating training images')
  img_batch, msk_batch = Augment(imgs=imgs_trn,msks= msks_trn,output_shape=config['output_shape'],\
                                 aug_ration=config['aug_ration'],transform=config['transform'],full_size=config['full_size'])
  print ('generating validation images')
  img_val, msk_val = Augment(imgs=imgs_val,msks= msks_val,output_shape=config['output_shape'],aug_ration=config['aug_ration'],\
                             transform=config['transform'],full_size=config['full_size'])
  imgs_tst=generate_test(config['test_path'],tiff_file=config['tiff_file'],height=config['output_shape'][0],width=config['output_shape'][1])
  if config['save']:
    save_npy(imgs_tst, config['test_image_filename'], config['save_path'])
    save_npy(img_batch,config['train_image_filename'],config['save_path'])
    save_npy(msk_batch, config['train_mask_filename'], config['save_path'])
    save_npy(img_val,config['validation_image_filename'],config['save_path'])
    save_npy(msk_val, config['validation_mask_filename'], config['save_path'])
    
    

In [19]:
"""

config={
'train_image_path':location of the training images
'train_mask_path':location of the training label
'full_size': If we need the maximum size
'tiff_file': To either load from tif images or other image formats
'aug_ration': Number of images to be generated from each image
'transforms': if the image needs to have additional transforms like flipping.
'test_path':location of testing file in case of tiff. Location of the test folder otherwise
'train_image_filename':name for the training images
'train_mask_filename':name for the training mask
'test_image_filename':name for testing images
'validation_image_filename':name for the validation images
'validation_mask_filename':'name for the validation masks
'save_path':loaction for saving the data
'save': if True saves the numpy files to the desired location.
'output_shape':(512,512) #if full size then irrelevent. Set fullsize is to true, If the image is not downsized.
'Size_of_test_image':(512,512)  Output size of the test image
}

#'train_image_path':'drive/My Drive/GAN_seg/ADDN-master/data/train-volume.tif',
#'train_mask_path':'drive/My Drive/GAN_seg/ADDN-master/data/train-labels.tif',
#'test_path':'drive/My Drive/GAN_seg/ADDN-master/data/test-volume.tif',
#'save_path':'drive/My Drive/GAN_seg/ADDN-master/',
"""


data={
      'train_image_path':'drive/My Drive/test/images',
      'train_mask_path':'drive/My Drive/test/label',
      'full_size':False,
      'tiff_file':False,
      'aug_ration': 100 ,
      'transform': True,
      
      'test_path':'drive/My Drive/test/test',
      'train_image_filename':'new_img_trn.npy',
      'train_mask_filename':'new_msk_trn.npy',
      'test_image_filename':'img_tst.npy',
      'validation_image_filename':'new_img_val.npy',
      'validation_mask_filename':'new_msk_val.npy',
      
      'save_path':'drive/My Drive/test/',
      'save':True,
      'output_shape':(128,128), #if full size then irrelevent fullsize is set to true.
      
}


main(data)

generating training images
original image number:0 output shape:(128,128)
original image number:1 output shape:(128,128)
original image number:2 output shape:(128,128)
original image number:3 output shape:(128,128)
original image number:4 output shape:(128,128)
original image number:5 output shape:(128,128)
original image number:6 output shape:(128,128)
original image number:7 output shape:(128,128)
original image number:8 output shape:(128,128)
original image number:9 output shape:(128,128)
original image number:10 output shape:(128,128)
original image number:11 output shape:(128,128)
original image number:12 output shape:(128,128)
original image number:13 output shape:(128,128)
original image number:14 output shape:(128,128)
original image number:15 output shape:(128,128)
original image number:16 output shape:(128,128)
original image number:17 output shape:(128,128)
original image number:18 output shape:(128,128)
original image number:19 output shape:(128,128)
original image number:2